In [1]:
from model import GPT
import numpy as np
import torch
from tqdm import tqdm
from copy import deepcopy
import tiktoken
from datasets import load_from_disk

In [2]:
model = GPT.from_pretrained('/home/mfx/huggingface/gpt2-xl')
device = "cuda:4"
model = model.to(device)
enc = tiktoken.get_encoding("gpt2")
test = load_from_disk("/home/mfx/CLOVer/output/wikitext-2-raw-v1")['test']
all_input_ids = torch.LongTensor(enc.encode("\n\n".join(test["text"]))).unsqueeze(0)
seq_len = all_input_ids.size(1)-1

loading weights from pretrained gpt: gpt2-xl
forcing vocab_size=50257, block_size=1024, bias=True
number of parameters: 1555.97M


In [3]:
def calculating_perplexity(model, datasest):
    max_length = model.config.block_size
    stride = 512
    nll_sum = 0.0
    n_tokens = 0
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = datasest[:, begin_loc:end_loc].to(device)
        target_ids = datasest[:, begin_loc+1:end_loc+1].to(device)
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            _,loss,_,_ = model(input_ids, targets=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = loss

        # Accumulate the total negative log-likelihood and the total number of tokens
        num_valid_tokens = (target_ids != -100).sum().item()  # number of valid tokens in target_ids
        batch_size = target_ids.size(0)
        num_loss_tokens = num_valid_tokens - batch_size  # subtract batch_size due to internal label shift
        nll_sum += neg_log_likelihood * num_loss_tokens

        n_tokens += num_loss_tokens

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    avg_nll = nll_sum / n_tokens  # average negative log-likelihood per token
    ppl = torch.exp(avg_nll)
    return avg_nll, ppl
    

In [4]:
calculating_perplexity(model, all_input_ids)

100%|█████████▉| 560/562 [01:04<00:00,  8.69it/s]


(tensor(2.6933, device='cuda:4'), tensor(14.7809, device='cuda:4'))

In [5]:
qk_norm_list = []
vo_norm_list = []
for name, module in model.named_modules():
    if name.endswith("attn"):
        q_weight = deepcopy(module.q_proj.weight.data) # (hidden_size, q_in_dim)
        q_bias = deepcopy(module.q_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
        q_weight = torch.cat([q_weight, q_bias],dim=1)  # (hidden_size, q_in_dim+1)
        k_weight = deepcopy(module.k_proj.weight.data) # (hidden_size, k_in_dim)
        k_bias = deepcopy(module.k_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
        k_weight = torch.cat([k_weight, k_bias],dim=1)  # (hidden_size, k_in_dim+1)
        qk_norm = q_weight.norm(p=2,dim=-1) * k_weight.norm(p=2,dim=-1)
        qk_norm_list.append(qk_norm)
        
        v_weight = deepcopy(module.v_proj.weight.data) # (hidden_size, v_in_dim)
        v_bias = deepcopy(module.v_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
        v_weight = torch.cat([v_weight, v_bias],dim=1)  # (hidden_size, v_in_dim+1)
        o_weight = deepcopy(module.o_proj.weight.data) # (o_out_dim,hidden_size)
        vo_norm = v_weight.norm(p=2,dim=-1) * o_weight.norm(p=2,dim=0)
        vo_norm_list.append(vo_norm)
qk_norm_list = torch.stack(qk_norm_list)
vo_norm_list = torch.stack(vo_norm_list)

In [5]:
for i in range(63):
    ratio = i/64
    qk_quantile = torch.quantile(qk_norm_list, ratio)
    vo_quantile = torch.quantile(vo_norm_list, ratio)
    print(ratio, qk_quantile, vo_quantile)
    for name, module in model.named_modules():
        if name.endswith("attn"):
            q_weight = deepcopy(module.q_proj.weight.data) # (hidden_size, q_in_dim)
            q_bias = deepcopy(module.q_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            q_weight = torch.cat([q_weight, q_bias],dim=1)  # (hidden_size, q_in_dim+1)
            k_weight = deepcopy(module.k_proj.weight.data) # (hidden_size, k_in_dim)
            k_bias = deepcopy(module.k_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            k_weight = torch.cat([k_weight, k_bias],dim=1)  # (hidden_size, k_in_dim+1)
            qk_norm = q_weight.norm(p=2,dim=-1) * k_weight.norm(p=2,dim=-1)
            module.q_proj.weight.data[qk_norm<qk_quantile]=0
            module.q_proj.bias.data[qk_norm<qk_quantile]=0
            module.k_proj.weight.data[qk_norm<qk_quantile]=0
            module.k_proj.bias.data[qk_norm<qk_quantile]=0
    print(calculating_perplexity(model, all_input_ids))
    

0.0 tensor(0.6197, device='cuda:4') tensor(0.1824, device='cuda:4')


  0%|          | 0/562 [00:00<?, ?it/s]

100%|█████████▉| 560/562 [01:03<00:00,  8.76it/s]


(tensor(2.6933, device='cuda:4'), tensor(14.7809, device='cuda:4'))
0.015625 tensor(1.5088, device='cuda:4') tensor(0.5361, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.64it/s]


...

(tensor(7.1653, device='cuda:4'), tensor(1293.8013, device='cuda:4'))
0.953125 tensor(4.7720, device='cuda:4') tensor(3.2120, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.68it/s]


(tensor(7.2911, device='cuda:4'), tensor(1467.1998, device='cuda:4'))
0.96875 tensor(5.0900, device='cuda:4') tensor(3.3482, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.68it/s]

(tensor(7.4612, device='cuda:4'), tensor(1739.2566, device='cuda:4'))


In [5]:
for i in range(63):
    ratio = i/64
    qk_quantile = torch.quantile(qk_norm_list, ratio)
    vo_quantile = torch.quantile(vo_norm_list, ratio)
    print(ratio, qk_quantile, vo_quantile)
    for name, module in model.named_modules():
        if name.endswith("attn"):
            v_weight = deepcopy(module.v_proj.weight.data) # (hidden_size, v_in_dim)
            v_bias = deepcopy(module.v_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            v_weight = torch.cat([v_weight, v_bias],dim=1)  # (hidden_size, v_in_dim+1)
            o_weight = deepcopy(module.o_proj.weight.data) # (o_out_dim,hidden_size)
            vo_norm = v_weight.norm(p=2,dim=-1) * o_weight.norm(p=2,dim=0)
            module.v_proj.weight.data[vo_norm<vo_quantile]=0
            module.v_proj.bias.data[vo_norm<vo_quantile]=0
            module.v_proj.weight.data[:,vo_norm<vo_quantile]=0
    print(calculating_perplexity(model, all_input_ids))
    

0.0 tensor(0.6197, device='cuda:4') tensor(0.1824, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.75it/s]


(tensor(2.6933, device='cuda:4'), tensor(14.7809, device='cuda:4'))
0.015625 tensor(1.5088, device='cuda:4') tensor(0.5361, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.63it/s]


(tensor(2.6959, device='cuda:4'), tensor(14.8191, device='cuda:4'))
0.03125 tensor(2.0249, device='cuda:4') tensor(0.7478, device='cuda:4')


...

(tensor(7.6285, device='cuda:4'), tensor(2055.9226, device='cuda:4'))
0.953125 tensor(4.7720, device='cuda:4') tensor(3.2120, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.64it/s]


(tensor(7.6285, device='cuda:4'), tensor(2055.9226, device='cuda:4'))
0.96875 tensor(5.0900, device='cuda:4') tensor(3.3482, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.64it/s]

(tensor(7.6285, device='cuda:4'), tensor(2055.9226, device='cuda:4'))


In [ ]:
for i in range(63):
    ratio = i/64
    qk_quantile = torch.quantile(qk_norm_list, ratio)
    vo_quantile = torch.quantile(vo_norm_list, ratio)
    print(ratio, qk_quantile, vo_quantile)
    for name, module in model.named_modules():
        if name.endswith("attn"):
            q_weight = deepcopy(module.q_proj.weight.data) # (hidden_size, q_in_dim)
            q_bias = deepcopy(module.q_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            q_weight = torch.cat([q_weight, q_bias],dim=1)  # (hidden_size, q_in_dim+1)
            k_weight = deepcopy(module.k_proj.weight.data) # (hidden_size, k_in_dim)
            k_bias = deepcopy(module.k_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            k_weight = torch.cat([k_weight, k_bias],dim=1)  # (hidden_size, k_in_dim+1)
            qk_norm = q_weight.norm(p=2,dim=-1) * k_weight.norm(p=2,dim=-1)
            module.q_proj.weight.data[qk_norm<qk_quantile]=0
            module.q_proj.bias.data[qk_norm<qk_quantile]=0
            module.k_proj.weight.data[qk_norm<qk_quantile]=0
            module.k_proj.bias.data[qk_norm<qk_quantile]=0
            v_weight = deepcopy(module.v_proj.weight.data) # (hidden_size, v_in_dim)
            v_bias = deepcopy(module.v_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            v_weight = torch.cat([v_weight, v_bias],dim=1)  # (hidden_size, v_in_dim+1)
            o_weight = deepcopy(module.o_proj.weight.data) # (o_out_dim,hidden_size)
            vo_norm = v_weight.norm(p=2,dim=-1) * o_weight.norm(p=2,dim=0)
            module.v_proj.weight.data[vo_norm<vo_quantile]=0
            module.v_proj.bias.data[vo_norm<vo_quantile]=0
            module.v_proj.weight.data[:,vo_norm<vo_quantile]=0
    print(calculating_perplexity(model, all_input_ids))
    

0.0 tensor(0.6197, device='cuda:4') tensor(0.1824, device='cuda:4')


100%|█████████▉| 560/562 [01:03<00:00,  8.77it/s]


(tensor(2.6933, device='cuda:4'), tensor(14.7809, device='cuda:4'))
0.015625 tensor(1.5088, device='cuda:4') tensor(0.5361, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.64it/s]


(tensor(2.8004, device='cuda:4'), tensor(16.4517, device='cuda:4'))
0.03125 tensor(2.0249, device='cuda:4') tensor(0.7478, device='cuda:4')


...

100%|█████████▉| 560/562 [01:04<00:00,  8.64it/s]


(tensor(6.8899, device='cuda:4'), tensor(982.2654, device='cuda:4'))
0.25 tensor(2.7462, device='cuda:4') tensor(1.4129, device='cuda:4')


100%|█████████▉| 560/562 [01:04<00:00,  8.63it/s]


(tensor(6.9386, device='cuda:4'), tensor(1031.3082, device='cuda:4'))
0.265625 tensor(2.7644, device='cuda:4') tensor(1.4589, device='cuda:4')


 67%|██████▋   | 377/562 [00:43<00:21,  8.64it/s]